## 3 Chamber Social Preference test - Behavioural analysis

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import ttest_ind, f_oneway

from shared.helper_functions import *

In [22]:
sns.set_theme()

Get the path to the folder that holds the behaviour data exported from BORIS software.

In [3]:
behaviour_data_dir = select_folder("Please select the sociability behaviour data directory")

If you want to save plots, select a directory you want to save them to.

In [4]:
plot_output_dir = select_or_create_folder("Please select or create a folder you wish to save the plots")

## Load the animal metadata

First, let's load the metadata that tells us which batch/cage combination is which animal (id).

In [5]:
all_animals_metadata = select_file("Please select the file holding metadata for all animals")
metadata = pd.read_excel(all_animals_metadata)
metadata

As the behaviour was tracked with Boris and saved to a separate file per animal, we load the files separately and merge them into one dataframe.

Note: animal from batch 5 cage 1 does not have an EEG file (transmitter was broken at the time), therefore there is not entry in the metadata file for that animal.

This means we have the following animals:

* batch1:
    * cage1: 78211 - 1.1
    * cage2: 78233 - 1.5
    * cage3: 78227 - 1.9
    * cage4: 78244 - 1.13
* batch2:
    * cage1: 79593 - 2.1
    * cage2: 79592 - 2.5
    * cage3: 79604 - 2.9
    * cage4: 79602 - 2.13
* batch4:
    * cage1: 80620 - 4.1
    * cage2: 80625 - 4.5
    * cage3: 80630 - 4.9
* batch5:
    * cage1: 81167 - 5.1 (no EEG)  
    * cage2: 81175 - 5.5
    * cage3: 81207 - 5.9
    * cage4: 81193 - 5.13
* batch5b:
    * cage1: 81217 - 5.17
    * cage2: 81218 - 5.21
* batch6:
    * cage1: 39489 - 6.1
    * cage2: 39508 - 6.5

With the above information we can create a dictionary that holds the batch-cage combination as keys, and the subject identifiers as values. These can be used to get the genotype that belongs to the individual animals.

In [6]:
subject_id_dict = {
    'batch1_cage1': 78211, 'batch2_cage1': 79593, 'batch5_cage1': 81167, 'batch4_cage1': 80620, 'batch5b_cage1': 81217, 'batch6_cage1': 39489,
    'batch1_cage2': 78233, 'batch2_cage2': 79592, 'batch5_cage2': 81175, 'batch4_cage2': 80625, 'batch5b_cage2': 81218, 'batch6_cage2': 39508,
    'batch1_cage3': 78227, 'batch2_cage3': 79604, 'batch5_cage3': 81207, 'batch4_cage3': 80630,
    'batch1_cage4': 78244, 'batch2_cage4': 79602, 'batch5_cage4': 81193,
}

## Data pre-processing

The data that is exported from the BORIS tracking application provides two rows per state-event. One represents the start of the event, and one the stop, each having a single timestamp.

We want to merge these rows and immediately calculate the duration of the state-event interaction. Let's do this first.

To do so, we define a function that merges some information from the START and the STOP rows.

In [7]:
def merge_event_rows(beh_data):
    merged_df = pd.concat([
        beh_data.iloc[::2].reset_index(drop=True),  # only keep each start row
        beh_data.iloc[::2].reset_index(drop=True)['Image index'].rename('Frame start'),  # interaction start frame, name it 'Frame start'
        beh_data.iloc[1::2].reset_index(drop=True)['Image index'].rename('Frame stop'),  # interaction stop frame, name it 'Frame stop'
        beh_data.iloc[1::2].reset_index(drop=True)['Time'] - beh_data.iloc[::2]['Time'].reset_index(drop=True),  # interaction duration
    ], axis=1)
    # rename the last column as it represents the duration of the interaction
    merged_df = merged_df.set_axis([*merged_df.columns[:-1], 'Interaction duration'], axis=1)
    # drop the columns we don't need
    cols_to_drop = ['Image index', 'Time', 'Observation type', 'Source', 'Time offset (s)', 'Subject', 'Comment', 'Image file path', 'Description', 'Behavioral category', 'Behavior type']
    return merged_df.drop(columns=cols_to_drop)

Then, we create one dataframe holding the behavioural data for all animals. Each row is one interaction event.

In [8]:
beh_df = pd.DataFrame()

# for each behaviour file (which holds behaviour data on one animal only)
for file in os.listdir(behaviour_data_dir):
    if not file.endswith('.xlsx') and 'batch' not in file:
        continue
        
    beh_dat = pd.read_excel(os.path.join(behaviour_data_dir, file))
    
    # get the subject id
    batch, cage = beh_dat['Observation id'].iloc[0].split('_')[:2]
    batch_cage = batch + '_' + cage
    subject_id = subject_id_dict[batch_cage]
    genotype = metadata[metadata.mouseId == subject_id]['genotype'].iloc[0]

    novel_cup_starts = beh_dat[(beh_dat['Behavior type'] == 'START') & (beh_dat['Behavior'] == 'novel_cup')]
    novel_cup_stops = beh_dat[(beh_dat['Behavior type'] == 'STOP') & (beh_dat['Behavior'] == 'novel_cup')]
    familiar_cup_starts = beh_dat[(beh_dat['Behavior type'] == 'START') & (beh_dat['Behavior'] == 'familiar_cup')]
    familiar_cup_stops = beh_dat[(beh_dat['Behavior type'] == 'STOP') & (beh_dat['Behavior'] == 'familiar_cup')]
    
    if not len(novel_cup_starts) == len(novel_cup_stops) or not len(familiar_cup_starts) == len(familiar_cup_stops):
        print(f'Number of event STARTs and STOPs do not match, skipping {beh_dat["Observation id"]} (id: {subject_id_dict[beh_dat["Observation id"]]})')
        continue
    
    # merge the start and stop rows and calculate some stuff (interaction duration etc)
    beh_data = merge_event_rows(beh_dat)
    beh_data.insert(1, 'subject_id', subject_id)
    beh_data.insert(2, 'genotype', genotype)
    beh_df = pd.concat([beh_df, beh_data], axis=0)

Let's inspect the dataframe

In [9]:
beh_df.head()

In [10]:
print(f'Subjects/animals present (n={len(np.unique(beh_df["subject_id"]))}):\n{np.unique(beh_df["subject_id"])} ')

## Social Preference metric calculations

In [12]:
behaviour_stats = pd.DataFrame()

# for each behaviour file (which holds behaviour data on one animal only)
for subject_id in np.unique(beh_df["subject_id"]):
    
    subject_data = beh_df[beh_df["subject_id"] == subject_id]
    
    mean_interaction_duration = np.mean(subject_data['Interaction duration'])
    interaction_ratio = len(subject_data[subject_data['Behavior'] == 'novel_cup']) / len(subject_data[subject_data['Behavior'] == 'familiar_cup'])
    total_interaction_duration_per_behavior = subject_data.groupby('Behavior')['Interaction duration'].sum()
    
    # Social Preference Index = (Time in chamber with novel conspecific - Time in chamber with familiar conspecific) / (Time in chamber with novel conspecific + Time in chamber with familiar conspecific)
    preference_index = (total_interaction_duration_per_behavior['novel_cup'] -  total_interaction_duration_per_behavior['familiar_cup']) /  (total_interaction_duration_per_behavior['novel_cup'] + total_interaction_duration_per_behavior['familiar_cup'])
    
    preference_index_n_interactions = (len(subject_data[subject_data['Behavior'] == 'novel_cup']) -  len(subject_data[subject_data['Behavior'] == 'familiar_cup'])) /  (len(subject_data[subject_data['Behavior'] == 'novel_cup']) + len(subject_data[subject_data['Behavior'] == 'familiar_cup']))
    
    temp = pd.DataFrame({
        'subject_id': subject_id,
        'genotype': subject_data['genotype'].iloc[0],
        'total_interactions': len(subject_data),
        'interaction_ratio': interaction_ratio,
        'total_interaction_time': np.sum(total_interaction_duration_per_behavior.values),
        'total_novel_interaction_duration': total_interaction_duration_per_behavior['novel_cup'],
        'total_familiar_interaction_duration': total_interaction_duration_per_behavior['familiar_cup'],
        'average_interaction_duration': mean_interaction_duration,
        'social_preference_index_interaction_duration': preference_index,
        'social_preference_index_n_interactions': preference_index_n_interactions,
    }, index=[0])
    behaviour_stats = pd.concat([behaviour_stats, temp], ignore_index=True)

Let's inspect these metrics a bit

In [13]:
behaviour_stats.head()

In [14]:
behaviour_stats.describe()

In [15]:
np.unique(behaviour_stats.genotype, return_counts=True)

Let's boxplot the calculated interaction metrics per genotype

In [26]:
behaviour_stats = behaviour_stats.sort_values(by='genotype', ascending=False)
boxplot_cols = behaviour_stats.columns[2:]

swarmplot_palette = {'DRD2-WT':'#AEC2B3', 'DRD2-KO':'#C68083'}
violin_palette = {'DRD2-WT':'#6A8D73', 'DRD2-KO':'#984447'}

title_label_dict = {
    'total_interactions': ('Total number of interactions with either cup', 'Count'),
    'total_interaction_time': ('Total interaction duration', 'Time (seconds)'),
    'interaction_ratio': ('Novel cup over familiar cup interaction ratio', 'Ratio'),
    'total_novel_interaction_duration': ('Total interaction time with novel cup', 'Time (seconds)'),
    'total_familiar_interaction_duration': ('Total interaction time with familiar cup', 'Time (seconds)'),
    'average_interaction_duration': ('Average interaction duration (both cups)', 'Duration (seconds)'),
    'social_preference_index_n_interactions': ('Sociability index (using total # of interactions)', 'Social preference index'),
    'social_preference_index_interaction_duration': ('Social preference index (using time spent interacting with cups)', 'Social preference index'),
}

fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(18, 8))
axs = axs.ravel()

for i, metric in enumerate(boxplot_cols):
    wt_metric_data = behaviour_stats[behaviour_stats.genotype == 'DRD2-WT'][metric]
    ko_metric_data = behaviour_stats[behaviour_stats.genotype == 'DRD2-KO'][metric]

    t_statistic, p_val = ttest_ind(wt_metric_data, ko_metric_data)
    
    sns.boxplot(data=behaviour_stats, x='genotype', hue='genotype', y=metric, ax=axs[i], linewidth=1.5, palette=violin_palette)
    sns.swarmplot(data=behaviour_stats, x='genotype', hue='genotype', y=metric, ax=axs[i], palette='dark:black')
    
    y_range = axs[i].get_ylim()
    offset = 0.05 * (y_range[1] - y_range[0])
    
    axs[i].text(0.5, y_range[1] - offset, f'T-stat: {round(t_statistic, 3)}\nP-value: {round(p_val, 3)}', ha='center', va='top', fontsize=9, fontweight='bold')
    axs[i].set_title(title_label_dict[metric][0])
    axs[i].set_ylabel(title_label_dict[metric][1])

fig.tight_layout()
plt.suptitle('Comparison of Social Preference Metrics in DRD2-WT and DRD2-KO Mice', fontsize=14, fontweight='bold', fontstyle='italic', y=1.025)
save_figure(os.path.join(plot_output_dir, 'boxplot_grid.pdf'))

In [18]:
for i, metric in enumerate(boxplot_cols):
    print(f'\n{metric}')
    wt_metric_data = behaviour_stats[behaviour_stats.genotype == 'DRD2-WT'][metric]
    ko_metric_data = behaviour_stats[behaviour_stats.genotype == 'DRD2-KO'][metric]
    
    # perform independent t-test to compare means
    result = ttest_ind(wt_metric_data, ko_metric_data)

    print("T-statistic:", result.statistic)
    print("P-value:", result.pvalue)
    
    f_statistic, p_value = f_oneway(wt_metric_data, ko_metric_data)

    print("F-statistic:", f_statistic)
    print("P-value:", p_value)

If there's any trend to be noticed, the KOs seem to be less social.

### Interaction duration distribution

In [19]:
beh_df.head()

Let's show the distribution of social, and non-social visits.

In [21]:
print(f'There are {len(beh_df[beh_df.Behavior == "novel_cup"])} novel cup observations, and {len(beh_df[beh_df.Behavior == "familiar_cup"])} familiar cup observations')

In [23]:
print(f'Total average interaction duration: {np.mean(beh_df["Interaction duration"])}')

So there's not that much difference in group size

In [22]:
fig = plt.figure(figsize=(12, 5))
sns.histplot(data=beh_df, x=beh_df['Interaction duration'], hue=beh_df.genotype, multiple='dodge', bins=[i + 0.5 for i in range(0, 19)])
plt.xticks(range(0, 20, 1))
plt.title("Distribution of interaction duration per genotype")
plt.xlabel("Interaction duration (seconds)")
plt.show()

In [24]:
fig = plt.figure(figsize=(12, 5))
sns.histplot(data=beh_df, x=beh_df['Interaction duration'], hue=beh_df.Behavior, multiple='dodge', bins=[i + 0.5 for i in range(0, 19)])
plt.xticks(range(0, 20, 1))
plt.title("Distribution of interaction duration per cup")
plt.xlabel("Interaction duration (seconds)")
plt.show()

### Interaction duration over time

Let us generate a grid plot that displays the interaction duration over the course of the experiment for each subject (10 minutes)

In [25]:
beh_df.head()

In [26]:
print(len(beh_df.subject_id.unique()))

Plot the interaction duration over the interaction number for one subject 

In [27]:
data = beh_df[beh_df.subject_id == 80630]

In [28]:
sns.lineplot(data=data, x=range(1,len(data)+1), y='Interaction duration', marker='o')
plt.title("Interaction duration for each interaction (true chronological order of interactions)")
plt.show()

And now with the start frame of the interaction on the x-axis

In [29]:
sns.lineplot(data=data, x='Frame start', y='Interaction duration', marker='o')
sns.scatterplot(data=data, x='Frame stop', y='Interaction duration', marker='o', color='orange')
plt.title("Interaction duration by the frame/time-point of the start of the scored interaction")
plt.show()

Now, let's plot this last one for all subjects in a grid

In [30]:
fig, axs = plt.subplots(nrows=5, ncols=4, figsize=(35, 30), sharey=True)
axs = axs.ravel()

for i, subj_id in enumerate(beh_df.subject_id.unique()):
    data = beh_df[beh_df.subject_id == subj_id]
    x = data['Frame start'] / 30 / 60  # to minutes
    x2 = data['Frame stop'] / 30 / 60
    sns.lineplot(data=data, x=x, y='Interaction duration', ax=axs[i], marker='o', label='Start interaction')
    sns.scatterplot(data=data, x=x2, y='Interaction duration', ax=axs[i], marker='o', color='orange', label='Stop interaction')
    axs[i].set_xlabel('Start of interaction (minutes)')
    axs[i].set_title(f'Subject id: {subj_id}, genotype: {data.genotype.iloc[0]}')
    plt.legend()

axs[-1].remove()
plt.tight_layout()
save_figure(os.path.join(plot_output_dir, f'behavioural_analysis/interaction_duration_trend_grid.pdf'))

Lets save a figure per subject that displays the trend of the interaction duration over the course of the experiment (10 minutes)

In [33]:
for i, subj_id in enumerate(beh_df.subject_id.unique()):
    data = beh_df[beh_df.subject_id == subj_id]
    sns.lmplot(data=data, x='Frame start', y='Interaction duration', aspect=1.5)
    plt.title(f'Interaction duration over the course of the experiment (Subject {subj_id}, genotype: {data.genotype.iloc[0]})')
    save_figure(os.path.join(plot_output_dir, f'behavioural_analysis/interaction_duration_trends/{subj_id}.pdf'))